## Count reads from the merged files

In [1]:
import os, glob, re, sys, gzip, time
import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio.Seq import Seq 
from pathlib import Path


from ngs_utils import * 

In [2]:
RUN_TO_COUNT=2

target_path = '../Data/designed_variants.csv'
path_prefix = f'../Data/run{RUN_TO_COUNT}/Parsed_data/'


merge_path=f'{path_prefix}/merged/'
output_path = f'{path_prefix}/library/'

Path(output_path).mkdir(parents=True, exist_ok=True)

In [3]:
T21_F = 'aaggtcatgattaca'.upper() #forward primer
T21_R = 'caagcagctaccgca'.upper() #reverse primer
MIN_QSCORE_THRESHOLD = 20

Count reads from all the lanes (This takes a few hours to run)

In [3]:
def count_reads(merge_path, output_path):
    if merge_path[-1] is not '/': merge_path += '/'
    if output_path[-1] is not '/': output_path += '/'
        
    sorted_out = output_path + 'sorted'
    discarded_out = output_path + 'discarded'
    lowQ_out = output_path + 'lowQ'
    
    check_directory_exists(sorted_out)
    check_directory_exists(lowQ_out)
    check_directory_exists(discarded_out)
    
    sample_files = glob.glob(merge_path + '*L1.fastq.gz') #use this for real data
                
    log = open(output_path + 'tabulate_output.txt', 'w')
    log.write('reads\t%good\t%bad\t%lowQ\t%!primer\tminQ\tname\n')
    
    for f in sample_files:
        sequences = {}
        sequences_lowQ = {}
        sequences_discarded = {}
        

        output_file = f.split('/')[-1].split('-')[0]+'.txt' # parse out the sample name (no lane numbers)

        count_finished = os.path.isfile(sorted_out + '/' + output_file)
        if count_finished:
            print('skipping ' + output_file)
        else:
            for lane_num in [1,2,3,4]:
                input_file = f.replace('L1.', 'L{num}.'.format(num=lane_num))            

                reads_processed_counts = 0
                good_read_counts = 0
                bad_read_counts_lowQ = 0
                bad_read_counts_no_primer = 0
                try:
                    print(input_file)
                    if ".gz" in input_file:
                        # print "gz in handle"
                        handle = gzip.open(input_file,"rt") # stream the gz file
                    else: 
                        print('handle is not gz')
                        handle = input_file
                    for read in SeqIO.parse(handle, 'fastq'):                        
    #                     if reads_processed_counts >= 100 : break # stop early                            
                        seq=str(read.seq.reverse_complement())
                        scores = read.letter_annotations["phred_quality"]
                        scores.reverse()

                        keep, library_seq, library_scores = get_libary_region(seq, scores, T21_F ,T21_R )

                        if keep:
                            min_qscore = min(library_scores)
                            if min_qscore > MIN_QSCORE_THRESHOLD:
                                tabulate(sequences, str(library_seq))                                
                                good_read_counts += 1
                            else:
                                tabulate(sequences_lowQ, str(library_seq))
                                bad_read_counts_lowQ += 1
                        else:
                            bad_read_counts_no_primer += 1
                            tabulate(sequences_discarded, str(library_seq))
                        reads_processed_counts += 1
                        if(reads_processed_counts%1e6==0):
                            print('-')
                        elif(reads_processed_counts%1e5==0):
                            print('.')
                    log.write('{p}\t{g:.2%}\t{b:.2%}\t{q:.2%}\t{f:.2%}\t{QC}\t{n}\n'.format(
                        p = reads_processed_counts, 
                        g = float(good_read_counts)/max(reads_processed_counts,1),     
                        b = float(bad_read_counts_lowQ+bad_read_counts_no_primer)/max(reads_processed_counts,1),
                        QC = MIN_QSCORE_THRESHOLD,
                        q = float(bad_read_counts_lowQ)/max(reads_processed_counts,1), 
                        f = float(bad_read_counts_no_primer)/max(reads_processed_counts,1),
                        n =  input_file))        
                except Exception as e:
                    log.write('ERROR\t\t\t\t\t\t' + input_file + '\n')
            write_seq_counts(sequences, sorted_out + '/' + output_file)
            write_seq_counts(sequences_lowQ, lowQ_out + '/' + output_file)
            write_seq_counts(sequences_discarded, discarded_out + '/' + output_file)
    log.close()             
sample_files=count_reads(merge_path, output_path)
    

../Data/run1/Parsed_data//library/sorted
../Data/run1/Parsed_data//library/lowQ
../Data/run1/Parsed_data//library/discarded
../Data/run1/Parsed_data//merged/S015_EK269_GAS1_p1_rep1c_plasmid-L1.fastq.gz
.
.
.
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S015_EK269_GAS1_p1_rep1c_plasmid-L2.fastq.gz
.
.
.
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S015_EK269_GAS1_p1_rep1c_plasmid-L3.fastq.gz
.
.
.
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S015_EK269_GAS1_p1_rep1c_plasmid-L4.fastq.gz
.
.
.
.
.
.
.
.
.
../Data/run1/Parsed_data//library/sorted/S015_EK269_GAS1_p1_rep1c_plasmid.txt
../Data/run1/Parsed_data//library/lowQ/S015_EK269_GAS1_p1_rep1c_plasmid.txt
../Data/run1/Parsed_data//library/discarded/S015_EK269_GAS1_p1_rep1c_plasmid.txt
../Data/run1/Parsed_data//merged/S042_EK269_GAS1_v3_rep1b_virus-L1.fastq.gz
.
.
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S042_EK269_GAS1_v3_rep1b_virus-L2.fastq.gz
.
.
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S042_EK269_GAS1_v3_rep1b_virus-L3.fastq.

.
.
.
.
.
.
.
.
../Data/run1/Parsed_data//library/sorted/S052_EK269_GAS1_v5_rep3d_virus.txt
../Data/run1/Parsed_data//library/lowQ/S052_EK269_GAS1_v5_rep3d_virus.txt
../Data/run1/Parsed_data//library/discarded/S052_EK269_GAS1_v5_rep3d_virus.txt
../Data/run1/Parsed_data//merged/S006_EK266_GAS1_p1_rep1b_plasmid-L1.fastq.gz
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S006_EK266_GAS1_p1_rep1b_plasmid-L2.fastq.gz
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S006_EK266_GAS1_p1_rep1b_plasmid-L3.fastq.gz
.
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S006_EK266_GAS1_p1_rep1b_plasmid-L4.fastq.gz
.
.
.
.
.
.
.
../Data/run1/Parsed_data//library/sorted/S006_EK266_GAS1_p1_rep1b_plasmid.txt
../Data/run1/Parsed_data//library/lowQ/S006_EK266_GAS1_p1_rep1b_plasmid.txt
../Data/run1/Parsed_data//library/discarded/S006_EK266_GAS1_p1_rep1b_plasmid.txt
../Data/run1/Parsed_data//merged/S050_EK269_GAS1_v5_rep3b_virus-L1.fastq.gz
.
.
.
.
.
.
../Data/run1/Parsed_data//merged/S050_EK269_GAS1_v5_rep3b_virus-L2.f

Combine reads from different replicas (This takes about an hour to run)

In [4]:
def combine_replicas(output_path):
    if output_path[-1] is not '/': output_path += '/'
    replica_files = glob.glob(output_path + 'sorted/*.txt')
    #print(replica_files)  
    for f in replica_files:

        sequences = {}
        output_file = f.split('.txt')[0][:]

        with open(f, 'r') as fh:
            lines = 0
            L = fh.readline() # skip the header
            for line in fh:
                parts=re.split('[,\t ]+', line.rstrip('\n'))
                seq = parts[0]
                count = int(parts[1])
                tabulate(sequences, seq, count)                                   
                lines += 1
#                     if lines > 10: break
        write_seq_counts(sequences, output_file + '.txt')
combine_replicas(output_path)

../Data/run1/Parsed_data//library/sorted/S050_EK269_GAS1_v5_rep3b_virus.txt
../Data/run1/Parsed_data//library/sorted/S015_EK269_GAS1_p1_rep1c_plasmid.txt
../Data/run1/Parsed_data//library/sorted/S051_EK269_GAS1_v5_rep3c_virus.txt
../Data/run1/Parsed_data//library/sorted/S014_EK269_GAS1_p1_rep1b_plasmid.txt
../Data/run1/Parsed_data//library/sorted/S005_EK266_GAS1_p1_rep1a_plasmid.txt
../Data/run1/Parsed_data//library/sorted/S006_EK266_GAS1_p1_rep1b_plasmid.txt
../Data/run1/Parsed_data//library/sorted/S044_EK269_GAS1_v3_rep1d_virus.txt
../Data/run1/Parsed_data//library/sorted/S013_EK269_GAS1_p1_rep1a_plasmid.txt
../Data/run1/Parsed_data//library/sorted/S048_EK269_GAS1_v4_rep2d_virus.txt
../Data/run1/Parsed_data//library/sorted/S052_EK269_GAS1_v5_rep3d_virus.txt
../Data/run1/Parsed_data//library/sorted/S016_EK269_GAS1_p1_rep1d_plasmid.txt
../Data/run1/Parsed_data//library/sorted/S042_EK269_GAS1_v3_rep1b_virus.txt
../Data/run1/Parsed_data//library/sorted/S047_EK269_GAS1_v4_rep2c_virus.txt


Build dataframe of counts for each sequence in the targeted designs. (a few minutes to run)

In [5]:
T21_WT = 'GACGAAGAGGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGTATGGTTCTGTATCTACCAACCTCCAGAGAGGCAACAGA'
T21_AA_WT ="DEEEIRTTNPVATEQYGSVSTNLQRGNR"
def reduce_library_to_target(output_path, target_path):  
    if output_path[-1] is not '/': output_path += '/'
    log_path = output_path + 'target_log.txt'
    sample_files = glob.glob(output_path + 'sorted/*.txt')
    sample_names = list()
    for s in sample_files:
        name = s.split('.txt')[0].split('/')[-1]
        name=name[name.index("_")+1:]
        sample_names.append(name)

    #return sample_names
    target = list()
    data = {}

    with open(log_path, 'w') as oh:
        target_df = pd.read_csv(target_path)
        lines = 0
        nt_replicas = {}
        aa_replicas = {}
        target = list()
        for i, row in target_df.iterrows():
            aa = row["aa"]
            aa_mask = row["aa_mask"]
            nt = row["nt"]
            category = row["category"]
            control_var = row["control_var"]
            rep_var = row["rep_var"]
            chip_var = row["chip_var"]
            is_wt = 1 if nt == T21_WT else 0
            is_aa_wt=1 if aa== T21_AA_WT else 0
            num_mut = sum([1 if x!='_' else 0 for x in aa_mask])            

            if(num_mut == 1 and len(aa_mask) == 29): # fix any instances where single insertions were not lowercase
                for i in range(len(aa)):
                    if(aa_mask[i]!='_'):
                        aa_mask = aa_mask.replace(aa_mask[i], aa_mask[i].lower())
                        aa = aa[:i] + aa_mask[i] + aa[i+1:]
            if nt not in nt_replicas: # count the aa replicas
                tabulate(aa_replicas, aa)
                tabulate(nt_replicas, nt)            
                target.append({'nt':nt,
                               'category':category,
                               'chip':chip_var,
                               'control':control_var,
                               'rep_original':rep_var,
                               'aa':aa, 
                               'mask':aa_mask, 
                               'mut':num_mut, 
                               'is_wt_nt':is_wt, 
                               'is_wt_aa':is_aa_wt,
                               'rep_i':aa_replicas[aa]})      

        print('number of sequences in target lib: ', len(target)+1)
        for i in range(len(target)):
            t = target[i]
            t['rep_total'] = aa_replicas[t['aa']]
        # add empty counts entries to data
        for i in range(len(target)):
            t = target[i]
            nt = t['nt']
            t.pop('nt') # remove it since we already will have this info in the key
            for n in sample_names: # set initial count for each sample to 0
                t[n] = 0 
            data[nt] = t    

        all_total = target_total = 0
        for s in sample_files:
            print(s)
            lines = all_counts = target_counts = 0
            name = s.split('.txt')[0].split('/')[-1]
            name=name[name.index("_")+1:]
            with open(s, 'r') as fh:            
                fh.readline() # skip the header
                for line in fh:
                    parts=re.split('[,\t ]+', line.rstrip('\n'))
                    seq = parts[0]
                    count = int(parts[1])                      
                    lines+=1
                    all_counts += count
                    if seq in data:
                        data[seq][name] = count
                        target_counts += count
                    if lines%300000==0: print ('.')
    #                     if lines > 100: break
                oh.write('{name}, {L:.2E}, {N:.2E}, {T:.2E}, {percent:.3}\n'.format(name=name, 
                                                               L=lines,
                                                               N=all_counts, 
                                                               T=target_counts, 
                                                               percent=float(target_counts)/all_counts)) 
                all_total += all_counts
                target_total += target_counts
            oh.write('{name}, {L:.2E}, {N:.2E}, {T:.2E}, {percent:.3}\n'.format(name='All', 
                                                           L=lines,
                                                           N=all_total, 
                                                           T=target_total, 
                                                           percent=float(target_total)/all_total)) 
    return data
        
        
    
start_time = time.time()
data = reduce_library_to_target(output_path, target_path)

number of sequences in target lib:  243484
../Data/run1/Parsed_data//library/sorted/S050_EK269_GAS1_v5_rep3b_virus.txt
../Data/run1/Parsed_data//library/sorted/S015_EK269_GAS1_p1_rep1c_plasmid.txt
.
.
.
../Data/run1/Parsed_data//library/sorted/S051_EK269_GAS1_v5_rep3c_virus.txt
../Data/run1/Parsed_data//library/sorted/S014_EK269_GAS1_p1_rep1b_plasmid.txt
.
.
../Data/run1/Parsed_data//library/sorted/S005_EK266_GAS1_p1_rep1a_plasmid.txt
.
.
../Data/run1/Parsed_data//library/sorted/S006_EK266_GAS1_p1_rep1b_plasmid.txt
.
.
../Data/run1/Parsed_data//library/sorted/S044_EK269_GAS1_v3_rep1d_virus.txt
../Data/run1/Parsed_data//library/sorted/S013_EK269_GAS1_p1_rep1a_plasmid.txt
.
.
../Data/run1/Parsed_data//library/sorted/S048_EK269_GAS1_v4_rep2d_virus.txt
../Data/run1/Parsed_data//library/sorted/S052_EK269_GAS1_v5_rep3d_virus.txt
../Data/run1/Parsed_data//library/sorted/S016_EK269_GAS1_p1_rep1d_plasmid.txt
.
.
../Data/run1/Parsed_data//library/sorted/S042_EK269_GAS1_v3_rep1b_virus.txt
../Data

In [6]:
df_data = pd.DataFrame(list(data.values()), index=list(data.keys()))

In [7]:
list(df_data.columns)

['EK266_GAS1_p1_rep1a_plasmid',
 'EK266_GAS1_p1_rep1b_plasmid',
 'EK269_GAS1_p1_rep1a_plasmid',
 'EK269_GAS1_p1_rep1b_plasmid',
 'EK269_GAS1_p1_rep1c_plasmid',
 'EK269_GAS1_p1_rep1d_plasmid',
 'EK269_GAS1_v3_rep1a_virus',
 'EK269_GAS1_v3_rep1b_virus',
 'EK269_GAS1_v3_rep1c_virus',
 'EK269_GAS1_v3_rep1d_virus',
 'EK269_GAS1_v4_rep2a_virus',
 'EK269_GAS1_v4_rep2b_virus',
 'EK269_GAS1_v4_rep2c_virus',
 'EK269_GAS1_v4_rep2d_virus',
 'EK269_GAS1_v5_rep3a_virus',
 'EK269_GAS1_v5_rep3b_virus',
 'EK269_GAS1_v5_rep3c_virus',
 'EK269_GAS1_v5_rep3d_virus',
 'aa',
 'category',
 'chip',
 'control',
 'is_wt_aa',
 'is_wt_nt',
 'mask',
 'mut',
 'rep_i',
 'rep_original',
 'rep_total']

In [8]:
print(len(df_data))
df_data.head()

243483


,EK266_GAS1_p1_rep1a_plasmid,EK266_GAS1_p1_rep1b_plasmid,EK269_GAS1_p1_rep1a_plasmid,EK269_GAS1_p1_rep1b_plasmid,EK269_GAS1_p1_rep1c_plasmid,EK269_GAS1_p1_rep1d_plasmid,EK269_GAS1_v3_rep1a_virus,EK269_GAS1_v3_rep1b_virus,EK269_GAS1_v3_rep1c_virus,EK269_GAS1_v3_rep1d_virus,...,category,chip,control,is_wt_aa,is_wt_nt,mask,mut,rep_i,rep_original,rep_total
GACGAAGATGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGTATGGTTCTGTATCTACCAACCTCGGGGGGGAAGGCGACAACAGA,51,44,36,58,77,37,195,227,144,168,...,rnn_designed_plus_rand_train_walked,1,0,0,0,__D____________________gGE_d__,5,1,1,1
GACGAGGACGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGTATGGTTCTGTATCTACCAACCTCCAGGATAACGGCAACAACGATAGA,4,7,6,8,8,8,16,17,16,12,...,rnn_designed_plus_rand_train_walked,1,0,0,0,__D_____________________Dn_n_d_,5,1,1,1
GACGAGGACGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGTATGGTGCGGTATCTACCAACCTCCAGGGCGACGGCAACGATAGA,23,11,11,12,13,13,36,47,34,50,...,rnn_designed_plus_rand_train_walked,1,0,0,0,__D______________A______Gd__d_,5,1,1,1
GACGAAGAGGAAATCGCTACAACCAATCCCGTGGCTACGGAGCAGTATGGTTCTGTATCTACCAACCTCCAGCACGACGGCGATGAAAGA,10,16,12,9,14,11,14,12,8,12,...,rnn_designed_plus_rand_train_walked,1,0,0,0,_____A__________________Hd_De_,5,1,1,1
GACGAACACGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGTATGGTAATGTATCTACCAACCTCCAGGGCGGAGGCGACAACAGA,27,20,12,11,37,32,76,64,63,50,...,rnn_designed_plus_rand_train_walked,1,0,0,0,__H______________N______Gg_d__,5,1,1,1


In [9]:
df_data.to_csv('../Data/raw_counts_NextSeq_run{RUN_TO_COUNT}.txt', index=True)
